In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from numba import jit

In [ ]:
@jit(nopython=True)
def initialize_lattice(size):
    return np.ones((size, size))  # Random initial spin configuration

In [ ]:
@jit(nopython=True)
def flip_cluster(lattice, beta):
    size = lattice.shape[0]
    x, y = np.random.randint(size), np.random.randint(size)
    spin = lattice[x, y]
    cluster = set([(x, y)])  # Set to store the cluster coordinates
    stack = [(x, y)]  # Stack for DFS

    while stack:
        i, j = stack.pop()
        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            nx, ny = (i + dx) % size, (j + dy) % size  # Periodic boundary conditions
            if (nx, ny) not in cluster and lattice[nx, ny] == spin and np.random.rand() < (1 - np.exp(-2 * beta)):
                stack.append((nx, ny))
                cluster.add((nx, ny))

    for i, j in cluster:
        lattice[i, j] *= -1  # Flip spins in the cluster

In [ ]:
@jit(nopython=True)
def monte_carlo_step(lattice, beta):
    flip_cluster(lattice, beta)


In [ ]:
@jit(nopython=True)
def average_magnetization(lattice):
    return np.abs(np.sum(lattice)) / lattice.size

In [ ]:
@jit(nopython=True)
def average_energy(lattice):
    size = lattice.shape[0]
    energy = 0
    for i in range(size):
        for j in range(size):
            spin = lattice[i, j]
            nb_sum = lattice[(i + 1) % size, j] + lattice[i, (j + 1) % size] + lattice[(i - 1) % size, j] + lattice[i, (j - 1) % size]
            energy += -nb_sum * spin
    return energy / size**2

In [ ]:
@jit(nopython=True)
def simulate_ising_model(size, temperature_range, num_steps):
    temperatures =temperature_range
    magnetizations = []
    energies = []
    heat_capacities = []
    suscpetibilities = []
    lattice = initialize_lattice(size)
    Lattice_profile = []


    for temperature in temperatures:
        beta = 1 / temperature
        magnetization_sum = 0
        energy_sum = 0
        magnetization_squared_sum = 0
        energy_squared_sum = 0

        for _ in range(num_steps):
            monte_carlo_step(lattice, beta)
            
            magnetization = average_magnetization(lattice)
            energy = average_energy(lattice)

            magnetization_sum += magnetization
            energy_sum += energy
            magnetization_squared_sum += magnetization ** 2
            energy_squared_sum += energy ** 2
            
        
        magnetizations.append(magnetization_sum / num_steps)
        energies.append(energy_sum / num_steps)
        heat_capacity = (energy_squared_sum / num_steps - (energy_sum / num_steps) ** 2) / (temperature ** 2)
        heat_capacities.append(heat_capacity)
        susceptibility = (magnetization_squared_sum / num_steps - (magnetization_sum / num_steps) ** 2) / (temperature)
        suscpetibilities.append(susceptibility)
        Lattice_profile.append(lattice.copy())
        print("TEMPERATURE =",temperature)

    return temperatures, magnetizations, energies, heat_capacities, suscpetibilities,Lattice_profile





In [ ]:
# Simulation parameters
size = 20
temperature_range = np.array([x*0.05 for x in range(1,101)])
num_steps = 1000

# Perform simulation
temperatures, magnetizations, energies, heat_capacities, susceptibilities, lattice_profile = simulate_ising_model(size, temperature_range, num_steps)

In [ ]:
t = np.array([x*0.5 for x in range(1,11)])
M = []
E = []
Cv = []
X = []
for i in range(len(t)):
    M.append(np.array(magnetizations[i*10:(i+1)*10]).mean())
    E.append(np.array(energies[i*10:(i+1)*10]).mean())
    Cv.append(np.array(heat_capacities[i*10:(i+1)*10]).mean())
    X.append(np.array(susceptibilities[i*10:(i+1)*10]).mean())
    
    

In [ ]:
len(M)

In [ ]:
# Plot results
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(temperatures, magnetizations, color='b')
plt.xlabel('Temperature')
plt.ylabel('Average Magnetization')

plt.subplot(2, 2, 2)
plt.plot(temperatures, energies, color='r')
plt.xlabel('Temperature')
plt.ylabel('Average Energy')

plt.subplot(2, 2, 3)
plt.plot(temperatures, heat_capacities, color='g')
plt.xlabel('Temperature')
plt.ylabel('Heat Capacity')

plt.subplot(2, 2, 4)
plt.plot(temperatures, susceptibilities , color='y')
plt.xlabel('Temperature')
plt.ylabel('Susceptibilities')

plt.tight_layout()
plt.show()

In [ ]:
# Plot results
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(temperatures, magnetizations, color='b')
plt.xlabel('Temperature')
plt.ylabel('Average Magnetization')

plt.subplot(2, 2, 2)
plt.plot(temperatures, energies, color='r')
plt.xlabel('Temperature')
plt.ylabel('Average Energy')

plt.subplot(2, 2, 3)
plt.plot(temperatures, heat_capacities, color='g')
plt.xlabel('Temperature')
plt.ylabel('Heat Capacity')

plt.subplot(2, 2, 4)
plt.plot(temperatures, susceptibilities , color='y')
plt.xlabel('Temperature')
plt.ylabel('Susceptibilities')

plt.tight_layout()
plt.show()

In [ ]:
# Plot results
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(t, M, color='b')
plt.xlabel('Temperature')
plt.ylabel('Average Magnetization')

plt.subplot(2, 2, 2)
plt.plot(t, E, color='r')
plt.xlabel('Temperature')
plt.ylabel('Average Energy')

plt.subplot(2, 2, 3)
plt.plot(t, Cv, color='g')
plt.xlabel('Temperature')
plt.ylabel('Heat Capacity')

plt.subplot(2, 2, 4)
plt.plot(t, X , color='y')
plt.xlabel('Temperature')
plt.ylabel('Susceptibilities')

plt.tight_layout()
plt.show()

In [ ]:
len(lattice_profile)

In [ ]:

def print_lattice(lattice):
    plt.imshow(lattice, cmap='viridis')
    plt.colorbar()
    plt.show()

In [ ]:
print_lattice(lattice_profile[0])

In [ ]:
Te = np.array([x/4 for x in range(1,21)])

In [ ]:
for i in range(0,len(lattice_profile),5):
    x = int(i/5)
    m = np.sum(np.array(lattice_profile[i]))/400
    if m >=0:
        print("Temperature =",Te[x],"Magnetization =",m)
        print_lattice(lattice_profile[i])
    else:
        print("Temperature =",Te[x],"Magnetization =",m*-1)
        print_lattice(lattice_profile[i]*-1)

# ****FOR 50x50 lattice

In [ ]:
# Simulation parameters
size = 50
temperature_range = np.array([x*0.05 for x in range(1,101)])
num_steps = 1000

# Perform simulation
temperatures50, magnetizations50, energies50, heat_capacities50, susceptibilities50, lattice_profile50 = simulate_ising_model(size, temperature_range, num_steps)

In [ ]:
# Plot results
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(temperatures, magnetizations50, color='b')
plt.xlabel('Temperature')
plt.ylabel('Average Magnetization')

plt.subplot(2, 2, 2)
plt.plot(temperatures, energies50, color='r')
plt.xlabel('Temperature')
plt.ylabel('Average Energy')

plt.subplot(2, 2, 3)
plt.plot(temperatures, heat_capacities50, color='g')
plt.xlabel('Temperature')
plt.ylabel('Heat Capacity')

plt.subplot(2, 2, 4)
plt.plot(temperatures, susceptibilities50, color='y')
plt.xlabel('Temperature')
plt.ylabel('Susceptibilities')

plt.tight_layout()
plt.show()

In [ ]:
t = np.array([x*0.5 for x in range(1,11)])
M50 = []
E50 = []
Cv50 = []
X50 = []
for i in range(len(t)):
    M50.append(np.array(magnetizations50[i*10:(i+1)*10]).mean())
    E50.append(np.array(energies50[i*10:(i+1)*10]).mean())
    Cv50.append(np.array(heat_capacities50[i*10:(i+1)*10]).mean())
    X50.append(np.array(susceptibilities50[i*10:(i+1)*10]).mean())
    
    

In [ ]:
# Plot results
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(t, M50, color='b')
plt.xlabel('Temperature')
plt.ylabel('Average Magnetization')

plt.subplot(2, 2, 2)
plt.plot(t, E50, color='r')
plt.xlabel('Temperature')
plt.ylabel('Average Energy')

plt.subplot(2, 2, 3)
plt.plot(t, Cv50, color='g')
plt.xlabel('Temperature')
plt.ylabel('Heat Capacity')

plt.subplot(2, 2, 4)
plt.plot(t, X50 , color='y')
plt.xlabel('Temperature')
plt.ylabel('Susceptibilities')

plt.tight_layout()
plt.show()

In [ ]:
for i in range(0,len(lattice_profile50),5):
    x = int(i/5)
    m=np.sum(np.array(lattice_profile50[i]))/2500
    if m >=0:
        print("Temperature =",Te[x],"Magnetization =",m)
        print_lattice(lattice_profile50[i])
    else:
        print("Temperature =",Te[x],"Magnetization =",m*-1)
        print_lattice(lattice_profile50[i]*-1)
    

# FOR 100x100****

In [ ]:
# Simulation parameters
size = 100
temperature_range = np.array([x*0.05 for x in range(1,101)])
num_steps = 1000

# Perform simulation
temperatures100, magnetizations100, energies100, heat_capacities100, susceptibilities100, lattice_profile100 = simulate_ising_model(size, temperature_range, num_steps)

In [ ]:
# Plot results
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(temperatures100, magnetizations100, color='b')
plt.xlabel('Temperature')
plt.ylabel('Average Magnetization')

plt.subplot(2, 2, 2)
plt.plot(temperatures100, energies100, color='r')
plt.xlabel('Temperature')
plt.ylabel('Average Energy')

plt.subplot(2, 2, 3)
plt.plot(temperatures100, heat_capacities100, color='g')
plt.xlabel('Temperature')
plt.ylabel('Heat Capacity')

plt.subplot(2, 2, 4)
plt.plot(temperatures100, susceptibilities100, color='y')
plt.xlabel('Temperature')
plt.ylabel('Susceptibilities')

plt.tight_layout()
plt.show()

In [ ]:
t = np.array([x*0.5 for x in range(1,11)])
M100 = []
E100 = []
Cv100 = []
X100 = []
for i in range(len(t)):
    M100.append(np.array(magnetizations100[i*10:(i+1)*10]).mean())
    E100.append(np.array(energies100[i*10:(i+1)*10]).mean())
    Cv100.append(np.array(heat_capacities100[i*10:(i+1)*10]).mean())
    X100.append(np.array(susceptibilities100[i*10:(i+1)*10]).mean())
    

In [ ]:
# Plot results
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(t, M100, color='b')
plt.xlabel('Temperature')
plt.ylabel('Average Magnetization')

plt.subplot(2, 2, 2)
plt.plot(t, E100, color='r')
plt.xlabel('Temperature')
plt.ylabel('Average Energy')

plt.subplot(2, 2, 3)
plt.plot(t, Cv100, color='g')
plt.xlabel('Temperature')
plt.ylabel('Heat Capacity')

plt.subplot(2, 2, 4)
plt.plot(t, X100 , color='y')
plt.xlabel('Temperature')
plt.ylabel('Susceptibilities')

plt.tight_layout()
plt.show()

In [ ]:
for i in range(0,len(lattice_profile100),5):
    x = int(i/5)
    m = np.sum(np.array(lattice_profile100[i]))/10000
    if m >=0:
        print("Temperature =",Te[x],"Magnetization =",m)
        print_lattice(lattice_profile100[i])
    else:
        print("Temperature =",Te[x],"Magnetization =",m*-1)
        print_lattice(lattice_profile100[i]*-1)

In [ ]:
plt.figure(figsize=(15, 18))

plt.subplot(4,2,1)
plt.title("Temperature vs Magnetization plot comparisons for different size of lattice",)
plt.xlabel("Temperature")
plt.ylabel("Average Magnetization")
plt.plot(temperatures, magnetizations,color = "r")
plt.plot(temperatures,magnetizations50,color ="b")
plt.plot(temperatures,magnetizations100,color = "g")
plt.legend(["20x20 Lattice","50x50 Lattice",'100x100 Lattice'])

plt.subplot(4,2,2)
plt.title("Temperature vs Magnetization plot comparisons for different size of lattice",)
plt.xlabel("Temperature")
plt.ylabel("Average Magnetization")
plt.plot(t, M,color = "r")
plt.plot(t,M50,color ="b")
plt.plot(t,M100,color = "g")
plt.legend(["20x20 Lattice","50x50 Lattice",'100x100 Lattice'])

plt.subplot(4,2,3)
plt.title("Temperature vs Energy plot comparisons for different size of lattice",)
plt.xlabel("Temperature")
plt.ylabel("Energy")
plt.plot(temperatures, energies,color = "r")
plt.plot(temperatures,energies50,color ="b")
plt.plot(temperatures,energies100,color = "g")
plt.legend(["20x20 Lattice","50x50 Lattice",'100x100 Lattice'])

plt.subplot(4,2,4)
plt.title("Temperature vs Energy plot comparisons for different size of lattice",)
plt.xlabel("Temperature")
plt.ylabel("Energy")
plt.plot(t, E,color = "r")
plt.plot(t,E50,color ="b")
plt.plot(t,E100,color = "g")
plt.legend(["20x20 Lattice","50x50 Lattice",'100x100 Lattice'])

plt.subplot(4,2,5)
plt.title("Temperature vs Heat Capacity plot comparisons for different size of lattice",)
plt.xlabel("Temperature")
plt.ylabel("Heat Capacity")
plt.plot(temperatures, heat_capacities,color = "r")
plt.plot(temperatures,heat_capacities50,color ="b")
plt.plot(temperatures,heat_capacities100,color = "g")
plt.legend(["20x20 Lattice","50x50 Lattice",'100x100 Lattice'])

plt.subplot(4,2,6)
plt.title("Temperature vs Heat Capacity plot comparisons for different size of lattice",)
plt.xlabel("Temperature")
plt.ylabel("Heat Capacity")
plt.plot(t, Cv,color = "r")
plt.plot(t,Cv50,color ="b")
plt.plot(t,Cv100,color = "g")
plt.legend(["20x20 Lattice","50x50 Lattice",'100x100 Lattice'])

plt.subplot(4,2,7)
plt.title("Temperature vs Susceptibility plot comparisons for different size of lattice",)
plt.xlabel("Temperature")
plt.ylabel("Susceptibity")
plt.plot(temperatures, susceptibilities,color = "r")
plt.plot(temperatures,susceptibilities50,color ="b")
plt.plot(temperatures,susceptibilities100,color = "g")
plt.legend(["20x20 Lattice","50x50 Lattice",'100x100 Lattice'])

plt.subplot(4,2,8)
plt.title("Temperature vs Susceptibility plot comparisons for different size of lattice",)
plt.xlabel("Temperature")
plt.ylabel("Susceptibity")
plt.plot(t, X,color = "r")
plt.plot(t,X50,color ="b")
plt.plot(t,X100,color = "g")
plt.legend(["20x20 Lattice","50x50 Lattice",'100x100 Lattice'])

plt.tight_layout()
plt.show()

In [ ]:
plt.title("Temperature vs Magnetization plot comparisons for different size of lattice",)
plt.xlabel("Temperature")
plt.ylabel("Average Magnetization")
plt.plot(temperatures, magnetizations,color = "r")
plt.plot(temperatures,magnetizations50,color ="b")
plt.plot(temperatures,magnetizations100,color = "g")
plt.legend(["20x20 Lattice","50x50 Lattice",'100x100 Lattice'])
plt.show()


In [ ]:
plt.title("Temperature vs Magnetization plot comparisons for different size of lattice",)
plt.xlabel("Temperature")
plt.ylabel("Average Magnetization")
plt.plot(t, M,color = "r")
plt.plot(t,M50,color ="b")
plt.plot(t,M100,color = "g")
plt.legend(["20x20 Lattice","50x50 Lattice",'100x100 Lattice'])
plt.show()

In [ ]:
plt.title("Temperature vs Energy plot comparisons for different size of lattice",)
plt.xlabel("Temperature")
plt.ylabel("Energy")
plt.plot(temperatures, energies,color = "r")
plt.plot(temperatures,energies50,color ="b")
plt.plot(temperatures,energies100,color = "g")
plt.legend(["20x20 Lattice","50x50 Lattice",'100x100 Lattice'])
plt.show()


In [ ]:
plt.title("Temperature vs Energy plot comparisons for different size of lattice",)
plt.xlabel("Temperature")
plt.ylabel("Energy")
plt.plot(t, E,color = "r")
plt.plot(t,E50,color ="b")
plt.plot(t,E100,color = "g")
plt.legend(["20x20 Lattice","50x50 Lattice",'100x100 Lattice'])
plt.show()

In [ ]:
plt.title("Temperature vs Heat Capacity plot comparisons for different size of lattice",)
plt.xlabel("Temperature")
plt.ylabel("Heat Capacity")
plt.plot(temperatures, heat_capacities,color = "r")
plt.plot(temperatures,heat_capacities50,color ="b")
plt.plot(temperatures,heat_capacities100,color = "g")
plt.legend(["20x20 Lattice","50x50 Lattice",'100x100 Lattice'])
plt.show()


In [ ]:
plt.title("Temperature vs Heat Capacity plot comparisons for different size of lattice",)
plt.xlabel("Temperature")
plt.ylabel("Heat Capacity")
plt.plot(t, Cv,color = "r")
plt.plot(t,Cv50,color ="b")
plt.plot(t,Cv100,color = "g")
plt.legend(["20x20 Lattice","50x50 Lattice",'100x100 Lattice'])
plt.show()

In [ ]:
plt.title("Temperature vs Susceptibility plot comparisons for different size of lattice",)
plt.xlabel("Temperature")
plt.ylabel("Susceptibity")
plt.plot(temperatures, susceptibilities,color = "r")
plt.plot(temperatures,susceptibilities50,color ="b")
plt.plot(temperatures,susceptibilities100,color = "g")
plt.legend(["20x20 Lattice","50x50 Lattice",'100x100 Lattice'])
plt.show()

In [ ]:
plt.title("Temperature vs Susceptibility plot comparisons for different size of lattice",)
plt.xlabel("Temperature")
plt.ylabel("Susceptibity")
plt.plot(t, X,color = "r")
plt.plot(t,X50,color ="b")
plt.plot(t,X100,color = "g")
plt.legend(["20x20 Lattice","50x50 Lattice",'100x100 Lattice'])
plt.show()

# WITH DIFFERENT PARAMETERS****

In [ ]:
# Simulation parameters
size = 20
temperature_range = np.array([0.5+x*0.3 for x in range(15)])
num_steps = 100000

# Perform simulation
temperatures_diff, magnetizations_diff, energies_diff, heat_capacities_diff, susceptibilities_diff, lattice_profile_diff = simulate_ising_model(size, temperature_range, num_steps)

In [ ]:
# Plot results
plt.figure(figsize=(25, 14))

plt.subplot(2, 2, 1)
plt.plot(temperatures_diff, magnetizations_diff,"bo-")
for x,y in zip(temperatures_diff,magnetizations_diff):
    label = "({:.1f},{:.2f})".format(x,y)
    plt.annotate(label, # this is the text
                 (x,y), # these are the coordinates to position the label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center

plt.xlabel('Temperature')
plt.ylabel('Average Magnetization')

plt.subplot(2, 2, 2)
plt.plot(temperatures_diff, energies_diff,"bo-")
for x,y in zip(temperatures_diff,energies_diff):
    label = "({:.1f},{:.2f})".format(x,y)
    plt.annotate(label, # this is the text
                 (x,y), # these are the coordinates to position the label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center

plt.xlabel('Temperature')
plt.ylabel('Average Energy')

plt.subplot(2, 2, 3)
plt.plot(temperatures_diff, heat_capacities_diff,"bo-")
for x,y in zip(temperatures_diff,heat_capacities_diff):
    label = "({:.1f},{:.2f})".format(x,y)
    plt.annotate(label, # this is the text
                 (x,y), # these are the coordinates to position the label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center
plt.xlabel('Temperature')
plt.ylabel('Heat Capacity')

plt.subplot(2, 2, 4)
plt.plot(temperatures_diff, susceptibilities_diff,"bo-")
for x,y in zip(temperatures_diff,susceptibilities_diff):
    label = "({:.1f},{:.2f})".format(x,y)
    plt.annotate(label, # this is the text
                 (x,y), # these are the coordinates to position the label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center
plt.xlabel('Temperature')
plt.ylabel('Susceptibilities')

plt.tight_layout()
plt.show()

In [ ]:
# Simulation parameters
size = 20
temperature_range = np.array([0.5+x*0.3 for x in range(15)])
num_steps = 100000

# Perform simulation
temperatures_diff_50, magnetizations_diff_50, energies_diff, heat_capacities_diff, susceptibilities_diff, lattice_profile_diff = simulate_ising_model(size, temperature_range, num_steps)